In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging
load_dotenv()
logging.langsmith("pr-jaunty-yak-21")

LangSmith 추적을 시작합니다.
[프로젝트명]
pr-jaunty-yak-21


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from operator import itemgetter
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/Introductory Statistics_230419_200206.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [7]:
store = {}

def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [9]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "anova가 뭐야?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123


'ANOVA(분산분석)는 세 개 이상의 모집단 평균이 서로 같은지 여부를 테스트하는 방법입니다. 이 방법은 다음과 같은 조건이 충족될 때 적용할 수 있습니다:\n- 모든 관심 있는 모집단이 정규 분포를 따릅니다.\n- 모집단의 표준 편차가 동일합니다.\n- 각 모집단에서 무작위로 독립적으로 선택된 샘플이 있습니다.\n- 독립 변수와 종속 변수가 각각 하나씩 있습니다.\n\nANOVA의 테스트 통계량은 F-비율(F-ratio)입니다.'

In [10]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "이전 답변을 영어로 번역해주세요."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123


'ANOVA (Analysis of Variance) is a method used to test whether the means of three or more populations are equal. This method can be applied when the following conditions are met:\n- All populations of interest follow a normal distribution.\n- The standard deviations of the populations are equal.\n- Samples are randomly and independently selected from each population.\n- There is one independent variable and one dependent variable.\n\nThe test statistic for ANOVA is the F-ratio.'

In [4]:
question = "t-test를 하는 이유가 뭐야?"
response = chain.invoke(question)
print(question)
print(response)

t-test를 하는 이유가 뭐야?
t-test를 하는 이유는 주로 두 가지입니다. 첫째, 두 개의 표본 평균이 통계적으로 유의미하게 다른지를 검정하기 위해서입니다. 둘째, 단일 모집단의 평균이 특정 값과 다른지를 검정하기 위해서입니다. t-test는 표본 크기가 작고 모집단의 표준 편차를 모를 때 사용됩니다.


In [5]:
question = "삼국지에서 적벽대전은 누가 이겼어?"
response = chain.invoke(question)
print(question)
print(response)

삼국지에서 적벽대전은 누가 이겼어?
모르겠어요.
